## Notebook to bulk download PlanetScope imagery through the API
Rainey Aberle, Fall 2021

Modified from [Planet Developers API Tutorial](https://developers.planet.com/docs/apis/data/) and Planet Labs GitHub Repository: [planetlabs/notebooks/jupyter-notebooks/orders/](https://github.com/planetlabs/notebooks/tree/master/jupyter-notebooks/orders)


__To-Do:__
- View image footprints before downloading.  
- Implement percent AOI coverage filter for image search

### 1. Define paths in directory and setup filters for image search (MODIFY THIS SECTION)

In [ ]:
# -----Define paths in directory
# path to planet-snow/
base_path = '/Users/raineyaberle/Research/PhD/Planet_snow_cover/planet-snow/' 
# path for saving image downloads
out_path = base_path+'../study-sites/Sperry/imagery/PlanetScope/'

# -----Area of Interest (AOI)
# Name of your polygon shapefile
# If your shapefile is not currently in this directory, you need to include the full file path in 'file_name' below
AOI_fn = base_path+'../../GIS_data/RGI_outlines/Sperry_RGI.shp'

# ----------Date Range----------
# Format: 'YYYY-MM-DD'
start_date = '2021-05-01'
end_date = '2021-10-31'

# ----------Cloud Cover Filter----------
# Format: decimal (e.g., 50% max cloud cover = 0.5)
max_cloud_cover = 0.2

# ----------Area Coverage (NOT CURRENTLY WORKING)---------- 
# Format: decimal (e.g., 50% min AOI bounding box area coverage = 0.5)
# min_area_coverage = 0.6

# ----------Item Type----------
# See here for possible image ("item") types:
# https://developers.planet.com/docs/apis/data/items-assets/
item_type = "PSScene4Band"

# ----------Asset Type----------
# Each Item Type has a number of asset types to choose from.
# Use the Item Type link above click on your Item Type to view the available Asset Types
asset_type = "analytic_sr"

# ----------AOI clipping----------
# Would you like to clip images to the AOI (True/False)?
# This greatly speeds up the ordering and downloading process.
clip_to_AOI = True

# ----------Harmonization----------
# option to harmonize PlanetScope imagery to Sentinel-2
harmonize = False # = True to harmonize

In [ ]:
# -----Import packages
import os
import json
import requests
from requests.auth import HTTPBasicAuth
from getpass import getpass
import geopandas as gpd
from shapely import geometry as sgeom
import rasterio as rio
import numpy as np
import matplotlib.pyplot as plt
import sys
# add path to functions
sys.path.insert(1, base_path+'functions/')
import orders_utils as orders
from IPython.display import Image

### 2. Reformat AOI for querying

In [ ]:
# -----Read in the shapefile
AOI = gpd.read_file(AOI_fn)
# Reproject to WGS84 if necessary
AOI = AOI.to_crs(4326)

# -----Convert AOI bounding box to geoJSON format
# Planet only excepts a bounding box as a spatial filter, 
# so we need to convert our AOI to a box (if it is not already). 
AOI_box = {u'type': u'Polygon',
            u'coordinates': [[
               [AOI.bounds.minx[0],AOI.bounds.miny[0]],
               [AOI.bounds.maxx[0],AOI.bounds.miny[0]],
               [AOI.bounds.maxx[0],AOI.bounds.maxy[0]],
               [AOI.bounds.minx[0],AOI.bounds.maxy[0]],
               [AOI.bounds.minx[0],AOI.bounds.miny[0]]
            ]]
          }
# Convert AOI geojson to a rasterio Shape for Quick Search function
AOI_box_shape = sgeom.shape(AOI_box)

# -----Plot AOI and bounding box
fig, ax1 = plt.subplots(1, 1, figsize=(8,8))
AOI.plot(ax=ax1) # AOI
ax1.plot(*AOI_box_shape.exterior.xy) # AOI box
ax1.set_title('AOI and bounding box')
plt.show()

### 3. Authentication via basic HTTP

Requires your Planet API Key. To find your API Key, Login to your account at [planet.com](https://www.planet.com/) and go to 'My Settings'. 

In [ ]:
# set API key as environment variable
print('Enter Planet API key:')
API_key = getpass()
os.environ['PL_API_KEY'] = API_key

# Setup the API Key stored as the `PL_API_KEY` environment variable
PLANET_API_KEY = os.getenv('PL_API_KEY')

# Orders URL
orders_url = 'https://api.planet.com/compute/ops/orders/v2'

# Authorize
auth = HTTPBasicAuth(PLANET_API_KEY, '')
response = requests.get(orders_url, auth=auth)
response

### 4. Compile filters to create a Quick Search request

Use Quick Search first to grab image IDs before ordering

In [ ]:
# -----Create request
QS_request = orders.build_QS_request(AOI_box_shape, max_cloud_cover, start_date, end_date, 
                        item_type, asset_type)

# -----Planet API Quick Search using created request
# fire off the POST request
QS_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=QS_request)
# Print resulting image IDs
im_ids = [feature['id'] for feature in QS_result.json()['features']]
im_ids.sort()
print(len(im_ids),'images found')

In [ ]:
# -----Filter images by area coverage (NOT WORKING)
# if min_area_coverage < 1.0:
#     items = orders.search_pl_api(QS_request, limit=1000)
#     # cache the overlaps as a list so we don't have to refetch items
#     overlaps = list(orders.get_overlap_shapes_utm(items, AOI_box_shape))
#     print(len(overlaps))

# display(orders.calculate_coverage(overlaps, (6,3), AOI_box_UTM_shape.bounds))

# def filter_by_coverage(overlaps, bounds):
#     im_ids_filtered = []
    
#     # get dimensions of coverage raster
#     mminx, mminy, mmaxx, mmaxy = bounds

#     y_count, x_count = dimensions
    
    
# AOI_box_UTM_shape.bounds

### 5. Place order and poll for success

- This section places the order and outputs the status of the order every ~10 sec. if successful. This can take a few minutes... 
- Wait until it outputs `success` to proceed to the next section. It will stop after 30 loops, so try rerunning the 'Poll for success' cell until seeing `success`.
- If you are ordering a LOT of images, consider narrowing your date range to download less images at a time. 

In [ ]:
# -----Build new request
request = orders.build_request_itemIDs(AOI_box, clip_to_AOI, harmonize, im_ids, item_type, asset_type)

# -----Place order
order_url = orders.place_order(orders_url, request, auth)

In [ ]:
# -----Poll for success
# Rerun this cell until it outputs "success"
orders.poll_for_success(order_url, auth)

In [ ]:
# -----View results
r = requests.get(order_url, auth=auth)
response = r.json()
results = response['_links']['results']
# print all files to be downloaded from order
[r['name'] for r in results]

### 6. Download each asset individually

In [ ]:
orders.download_results(results, out_path)

### _Optional:_ Plot downloaded images

In [ ]:
import glob 

# -----Grab output image file names
# set image output folder
im_path = out_path + '2020/PSScene4Band/'
# change directory to im_path
os.chdir(im_path) 
# grab image file names
im_fns = glob.glob('*_SR_*.tif')

# -----Loop through files
for im_fn in im_fns:
    
    # open image
    im = rio.open(im_fn)
    
    # read bands
    b = im.read(1).astype(float) / 10000
    r = im.read(2).astype(float) / 10000
    g = im.read(3).astype(float) / 10000
    nir = im.read(4).astype(float) / 10000
    
    # plot
    fig, ax = plt.subplots(1, 1, figsize=(6,6))
    ax.imshow(np.dstack([r, g, b]))
    ax.set_title(im_fn[0:8])
    plt.show()